In [4]:
chrom = '22' #sys.argv[1]
data_dir = '../split_gen' #sys.argv[2]

variant_file = '%s/chr.%s.gen.variants.txt.gz' % (data_dir, chrom)

In [7]:
import gzip
import numpy as np

# pull snp, del indices and positions
positions = []
snp_indices = []
del_indices = []
del_lengths = []
with gzip.open(variant_file, 'rt') as f:
    for i, line in enumerate(f):
        pieces = line.strip().split('\t')
        ref, alt = pieces[3], pieces[4]
        if len(ref) == 1 and len(alt) == 1 and ref != '.' and alt != '.':
            snp_indices.append(i)
        elif len(ref) > len(alt) and len(alt) == 1 and ref != '.' and alt != '.':
        	del_indices.append(i)
        	del_lengths.append(len(ref)-1)
        positions.append(int(pieces[1]))
positions = np.array(positions)
snp_positions = positions[snp_indices]

del_positions = set()
for index, length in zip(del_indices, del_lengths):
	start_pos = positions[index]+1
	end_pos = start_pos+length
	del_positions.update(range(start_pos, end_pos))
del_positions = np.asarray(sorted(del_positions))

print('Deleted positions', len(del_positions), 'SNP positions', len(snp_positions))

Deleted positions 326891 SNP positions 1004649


In [11]:
from os import listdir
from scipy import sparse

# load genotype data
gen_files = sorted([f for f in listdir(data_dir) if ('chr.%s' % chrom) in f and 'gen.npz' in f])
gen_file = gen_files[0]
start_index = 0

subchrom = sparse.load_npz('%s/%s' % (data_dir, gen_file))
m, n = subchrom.shape
start_pos, end_pos = positions[start_index], positions[-1]+1 if start_index+n+1 >= positions.shape[0] else positions[start_index+n+1]
print('start', start_pos, 'end', end_pos)

sub_snppos = snp_positions[(snp_positions>=start_pos) & (snp_positions < end_pos)].tolist()
sub_delpos = del_positions[(del_positions>=start_pos) & (del_positions < end_pos)].tolist()
ordered_positions = sorted(set(sub_snppos + sub_delpos))
pos_to_index = dict([(x, i) for i, x in enumerate(ordered_positions)])
old_n, n = n, len(ordered_positions)
print('m', 'old_n', old_n, 'new_n', n)

old_snp_indices = [x for x in snp_indices if positions[x] >= start_pos and positions[x] < end_pos]
new_snp_indices = [pos_to_index[positions[x]] for x in old_snp_indices]

full_subchrom = -np.ones((m, n), dtype=np.int8)

start 16050036 end 51244530
m old_n 1096535 new_n 1329620


KeyboardInterrupt: 

In [ ]:
# fill in snps
full_subchrom[:, new_snp_indices] = subchrom[:, old_snp_indices].A

In [ ]:
# fill in dels


In [ ]:
import matplotlib.pyplot as plt

plt.subplot(2, 1, 1)
plt.imshow(chrom_interest.A[:, is_del], aspect='auto', vmax=2, vmin=-1)
plt.subplot(2, 1, 2)
plt.imshow(chrom_interest.A[:, ~np.asarray(is_del)], aspect='auto', vmax=2, vmin=-1)

plt.show()